# Installing "Local" PySpark on Windows 10

1. Install Java 1.8 from Software Center
2. Install Python
3. Create pyspark_dev virtual environment
4. Activate "pyspark_dev" environment, then: pip install pyspark[sql] ipykernel
5. Install kernel: python -m ipykernel install --user --name pyspark_dev --display-name "Python (pyspark_dev)"
6. Set environment variables: PYSPARK_PYTHON=[path_to_python.exe] and SPARK_HOME=[path_to_site_packages/pyspark_folder]
7. Download winutils.exe from https://github.com/cdarlint/winutils, save locally to "hadoop/bin" folder and then
8. set HADOOP_HOME=[path_to_hadoop_folder]
9. Activate jupyterlab environment and then launch: jupyter lab

See this article https://phoenixnap.com/kb/install-spark-on-windows-10

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

22/01/02 20:21:59 WARN Utils: Your hostname, asus-laptop resolves to a loopback address: 127.0.1.1; using 172.30.59.173 instead (on interface eth0)
22/01/02 20:21:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/02 20:22:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv('data/cars.csv', header=True, sep=";")

In [3]:
df.show(5)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 5 rows

